In [ ]:
#!pip install pandas
#!pip install geopandas
#!pip install folium

In [ ]:
import os
import json
from urllib import request

import pandas as pd
import geopandas as gpd

import folium
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [ ]:
path = os.getcwd()
os.chdir("../")
# импорт функции rename_subj для приведения всех названий регионов к единому стандарту
from Utils.subj_rename import rename_subj
# импорт функции, сохраняющей карту в хорошем качестве в png формат
from Utils.save_map import map_to_png
os.chdir(path.split("\\")[-1])

# Обработка данных

In [ ]:
# импорт словаря переименований регионов для унифицирования их названия в используемых df
with open("../Utils/subj_rename.json", encoding="utf-8") as file:
    subj_rename_data = json.load(file)

In [ ]:
df_divorce = pd.read_excel("divorce.xls")
df_divorce["region"] = df_divorce["region"].str.strip()

# приведение названий регионов к единому стандарту для корректного отображения на карте
df_divorce = rename_subj(
    df_divorce,
    "region",
    subj_rename_data
)
display(df_divorce.head())
df_divorce.shape

In [ ]:
df_regions = gpd.read_file("../Utils/geo_data.json").rename(columns={"name": "region"})

# приведение названий регионов к единому стандарту для корректного отображения на карте
df_regions = rename_subj(
    df_regions,
    "region",
    subj_rename_data
)
display(df_regions.head())
df_regions.shape

In [ ]:
# Соединение датафреймов для отсеивания лишних полей по столбцу region
df_show = df_divorce.merge(df_regions, on="region", how="right")
display(df_show.head())
df_show.shape

In [ ]:
df_show[df_show.k.isna()]

# Визуализация

In [ ]:
# [64.6863136, 97.7453061] # широта и долгота России
Map = folium.Map(
    [69, 96],
    zoom_start=3.5,
    width="%100",
    height="%100"
)

folium.Choropleth(
    geo_data=df_regions,
    data=df_show,
    name='Разводы',
    columns=['region', 'k'],   
    legend_name='Коэффициент разводимости на 1000 человек',
    key_on='feature.properties.region',
    fill_color="PuBu",
    fill_opacity=0.8,
    line_opacity=0.5,
).add_to(Map)
folium.LayerControl().add_to(Map)

tooltip = "Click Here For More Info"
# координаты середины канала - 66°25' с. ш. 94°15' в. д.
marker = folium.Marker(
    location=[66.25, 91.15],
    popup="""
        <stong>
            <a href='https://t.me/maps_stat'>tg: https://t.me/maps_stat</a><br />
            <a href='https://vk.com/maps_stat'>vk: https://vk.com/maps_stat</a>
        </stong>
    """,
    icon=folium.DivIcon(
        html="""
            <div style="color:#878787;background:#878787:90px;text-align:center;"><h5><u>@maps_stat</u><h5></div>
        """
    ),
    fill_opacity=0.5
)
marker.add_to(Map)
Map

# Сохранение карты

In [ ]:
#!pip install pillow
#!pip install selenium

In [ ]:
# в html-формат
Map.save("divorce.html")

In [ ]:
# загрузка конфига
# нужен для корректной работы функции сохранения карты в виде изображения
with open("../config.json") as file:
    config = json.load(file)

In [ ]:
%%time
map_to_png(Map, config)